# 🧠 O-ISAC CoT Master Pipeline

Tek notebook ile tüm extraction pipeline'ı çalıştır.

**Aşamalar:**
1. 📦 Setup & Mount
2. 🏭 Phase 1: Data Prep (PDF → Markdown)
3. 🖼️ Phase 2: Visual Analysis
4. 🧠 Phase 3: CoT Extraction
5. 📊 Results & Export

**Gereksinimler:**
- Colab GPU Runtime (T4 veya A100)
- GROQ_API_KEY (Colab Secrets'da ayarlı)

---
**Son Güncelleme:** 2025-12-11
**Versiyon:** 1.0

---
## 📦 Section 1: Setup & Mount

In [ ]:
# @title 1.1 Install Dependencies
# Phase 1 & 2 heavy dependencies
!pip install marker-pdf -q
!pip install transformers torch pillow -q

# Phase 3 light dependencies
!pip install groq nest_asyncio pandas pyyaml -q

print("✅ Tüm bağımlılıklar yüklendi!")

In [ ]:
# @title 1.2 Mount Google Drive & Setup Paths
from google.colab import drive
from google.colab import userdata
import os
import sys

# Mount Drive
drive.mount('/content/drive')

# Project Paths
PROJECT_ROOT = '/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST'
NOTEBOOKS_DIR = os.path.join(PROJECT_ROOT, 'analysis/notebooks')
COT_LAB_DIR = os.path.join(PROJECT_ROOT, 'analysis/cot_laboratory')
PDF_DIR = os.path.join(PROJECT_ROOT, 'data/retrieved_docs')
MARKDOWN_DIR = os.path.join(PROJECT_ROOT, 'data/processed_markdowns')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'data/extraction_results_v3')

# Add to Python Path
sys.path.insert(0, NOTEBOOKS_DIR)
sys.path.insert(0, PROJECT_ROOT)

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📄 PDF Directory: {PDF_DIR}")
print(f"📝 Markdown Directory: {MARKDOWN_DIR}")
print(f"📊 Output Directory: {OUTPUT_DIR}")
print("✅ Paths configured!")

In [ ]:
# @title 1.3 Load API Key
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    print("✅ GROQ_API_KEY yüklendi!")
except Exception as e:
    print("❌ HATA: Sol menüden 🔑 Secrets bölümüne GROQ_API_KEY ekleyin!")
    print(f"   Hata detayı: {e}")

---
## 🏭 Section 2: Phase 1 - Data Prep (PDF → Markdown)

**⚠️ GPU Gerektirir!** Bu adım PDF'leri OCR ile markdown'a çevirir.

In [ ]:
# @title 2.1 Import Pipeline & Check Status
from extraction_pipeline_v3 import Config, CheckpointManager, phase1_marker_conversion

# Initialize
Config.init_dirs()
checkpoint = CheckpointManager(Config.CHECKPOINT_FILE)

# Show current status
processed = checkpoint.data.get('processed', {})
print(f"📊 Mevcut durum: {len(processed)} paper işlenmiş")
print(f"📂 PDF'ler: {PDF_DIR}")

# List PDFs
import glob
pdfs = glob.glob(os.path.join(PDF_DIR, '*.pdf'))
print(f"📄 Toplam PDF: {len(pdfs)}")

In [ ]:
# @title 2.2 Run PDF → Markdown Conversion (Phase 1)
# ⚠️ Bu adım uzun sürebilir (paper başına ~1-2 dk)

print("⏳ Phase 1: PDF → Markdown dönüşümü başlıyor...")
phase1_marker_conversion(checkpoint, force_all=False)
print("✅ Phase 1 tamamlandı!")

---
## 🖼️ Section 3: Phase 2 - Visual Analysis

BLIP ve DePlot modelleri ile görsel analiz yapılır.

In [ ]:
# @title 3.1 Run Visual Analysis (Phase 2)
from extraction_pipeline_v3 import phase2_visual_analysis

print("⏳ Phase 2: Görsel analiz başlıyor...")
phase2_visual_analysis(checkpoint)
print("✅ Phase 2 tamamlandı!")

---
## 🧠 Section 4: Phase 3 - CoT Extraction

Chain-of-Thought extraction ile yapısal veri çıkarma.

In [ ]:
# @title 4.1 Import CoT Laboratory
sys.path.insert(0, COT_LAB_DIR)
from core.assembler import CoTAssembler
from core.batch_runner import CoTFactory

# Default Recipe
RECIPE_PATH = 'analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml'

print("✅ CoT Laboratory yüklendi!")
print(f"📜 Recipe: {RECIPE_PATH}")

In [ ]:
# @title 4.2 Single Paper Test
# Önce tek bir paper üzerinde test et

TEST_PAPER_ID = "O_ISAC_029"  # @param {type:"string"}

import json

# Find paper markdown
paper_path = os.path.join(MARKDOWN_DIR, TEST_PAPER_ID, TEST_PAPER_ID, f"{TEST_PAPER_ID}.md")
vis_path = os.path.join(MARKDOWN_DIR, TEST_PAPER_ID, TEST_PAPER_ID, "visual_analysis.txt")

print(f"📄 Paper: {paper_path}")
print(f"   Exists: {os.path.exists(paper_path)}")

# Read content
with open(paper_path, 'r', encoding='utf-8') as f:
    content = f.read()

# Read visual if exists
visual_content = None
if os.path.exists(vis_path):
    with open(vis_path, 'r', encoding='utf-8') as f:
        visual_content = f.read()
    print(f"🖼️ Visual Analysis: {len(visual_content)} chars")

# Run extraction
assembler = CoTAssembler(PROJECT_ROOT)
result = assembler.run_extraction(
    RECIPE_PATH,
    content,
    paper_id=TEST_PAPER_ID,
    visual_content=visual_content
)

# Show result
if result['status'] == 'success':
    print("\n✅ EXTRACTION BAŞARILI!")
    print("\n📋 Reasoning Trace:")
    trace = result['parsed_output'].get('reasoning_trace', [])
    for step in trace:
        print(f"  {step.get('key')}: {step.get('value')[:80]}...")
else:
    print(f"\n❌ HATA: {result.get('error_message')}")

In [ ]:
# @title 4.3 Batch Extraction (Tüm Paper'lar)
# ⚠️ Bu uzun sürecek! ~2-3 dk per paper

RUN_BATCH = False  # @param {type:"boolean"}

if RUN_BATCH:
    factory = CoTFactory(PROJECT_ROOT)
    factory.run_batch(RECIPE_PATH)
else:
    print("ℹ️ Batch mode kapalı. Çalıştırmak için RUN_BATCH = True yapın.")

---
## 📊 Section 5: Results & Export

In [ ]:
# @title 5.1 View Latest Logs
import glob

logs_dir = os.path.join(COT_LAB_DIR, 'logs')
log_files = sorted(glob.glob(os.path.join(logs_dir, '*_RESULT.json')))[-5:]

print(f"📋 Son 5 extraction log:")
for log in log_files:
    filename = os.path.basename(log)
    # Parse: 20251210_144637_O_ISAC_029_llama-3.3-70b-versatile_RESULT.json
    parts = filename.split('_')
    date = parts[0]
    paper_id = f"{parts[2]}_{parts[3]}_{parts[4]}"
    print(f"  • {date}: {paper_id}")

In [ ]:
# @title 5.2 Export to CSV (TODO)
# Bu fonksiyon tüm log JSON'larını birleştirip CSV'ye çevirecek

print("📊 CSV export fonksiyonu henüz implemente edilmedi.")
print("   Sonuçlar logs/ klasöründe JSON olarak mevcut.")

---
## ✅ Done!

**Sonraki Adımlar:**
1. Single paper test sonuçlarını kontrol et
2. Kalite iyi ise batch mode'u aç
3. Tüm paper'ları işle
4. CSV export yap